In [1]:
using Plots
using Serialization
include("../interactions.jl")
include("../utils.jl")
In, coag_In = deserialize("../wikivote/wikivote_coag.data");

In [2]:
parents = [node.parent.index for node in In.nodes]
num_children = [length(node.children) for node in coag_In.nodes]
degrees = In.degrees
coag_degrees = coag_In.degrees;

In [4]:
include("../pdgm_coag_icrp.jl")
num_chains = 3
chains = Vector{PDGMCoagICRPChain}(undef, 0)
preds = Vector{PDGMCoagICRPPred}(undef, 0)
for i in 1:num_chains
    chain, pred = deserialize("results/pdgm_coag_icrp/run$i.data")
    push!(chains, chain)
    push!(preds, pred)
end

In [5]:
pred_parents = Vector{Vector{Int}}(undef, 0)
pred_num_children = Vector{Vector{Int}}(undef, 0)
pred_degrees = Vector{Vector{Int}}(undef, 0)
pred_coag_degrees = Vector{Vector{Int}}(undef, 0)
for i in 1:num_chains
    append!(pred_parents, preds[i].parents)
    append!(pred_num_children, preds[i].num_children)
    append!(pred_degrees, preds[i].degrees)
    append!(pred_coag_degrees, preds[i].coag_degrees)
end

In [6]:
kss = zeros(length(pred_num_children))
for i in 1:length(pred_num_children)
    kss[i] = ApproximateTwoSampleKSTest(num_children, pred_num_children[i]).δ
end

In [14]:
import Measures:mm
p = plot_fof(num_children)
plot_fof_CI(pred_num_children; redraw=true)
mean_ks, std_ks = round(mean(kss);digits=4), round(std(kss);digits=4)
title = L"Number of Children $D_{\mathrm{KS}}: %$(mean_ks) \pm %$(std_ks)$"
p = plot(p, size=(500, 400), margin=4mm, title=title)
savefig(p, "figures/pdgm_coag_icrp/num_children.pdf")

"c:\\workspace\\codes\\julia\\icrp.jl\\wikivote\\figures\\pdgm_coag_icrp\\num_children.pdf"

In [9]:

function compute_mean_coag_degree(
    degrees_list,
    coag_degrees_list,
    parents_list;    
    xloglim::Int=16,    
)

    edgebins = [2^i for i in 0:xloglim]
    xs = zeros(0)
    ys = [zeros(0) for _ in 1:length(degrees_list)]
    for (i, (lb, ub)) in enumerate(zip(edgebins[1:end-1], edgebins[2:end]))    
        cnts = zeros(0)
        for (degrees, coag_degrees, parents) in zip(degrees_list, coag_degrees_list, parents_list)
            inds = parents[(degrees .>= lb) .&& (degrees .<= ub)]   
            cnt = mean(coag_degrees[inds])            
            if cnt > 0
                push!(cnts, cnt)                
            end
        end

        if length(cnts) == length(degrees_list)
            push!(xs, 0.5*(lb+ub))
            for (j, cnt) in enumerate(cnts)
                push!(ys[j], cnt)
            end
        end

    end

    return xs, ys

end

oxs, oys = compute_mean_coag_degree([In.degrees], [coag_In.degrees], [parents])
xs, ys = compute_mean_coag_degree(pred_degrees, pred_coag_degrees, pred_parents)
ys = cat(ys...; dims=2)';

In [10]:
kss = zeros(size(ys, 1))
for i in 1:length(size(ys,1))
    kss[i] = ApproximateTwoSampleKSTest(oys[1], ys[i,:]).δ
end
println(mean(kss))

0.000303030303030303


In [13]:
p = plot(oxs, oys, xaxis=:log, yaxis=:log, 
            fillcolor="skyblue", markershape=:circle, 
            markersize=5, label=nothing, linecolor="steelblue", linewidth=1.5)


α = 0.95
lbs = zeros(length(xs))
ubs = zeros(length(xs))
for i in 1:length(xs)
    lbs[i], ubs[i] = quantile(ys[:,i], [0.5 * (1 - α), 0.5 * (1 + α)])
end

p = plot!(xs, lbs, fillrange=ubs, label=nothing, linecolor=nothing, 
        fillcolor="skyblue", fillalpha=0.3, xaxis=:log, yaxis=:log)

p = plot(p, size=(500, 400), margin=4mm, title="Average parent degrees")
savefig("figures/pdgm_coag_icrp/avg_parent_degrees.pdf")

            

"c:\\workspace\\codes\\julia\\icrp.jl\\wikivote\\figures\\pdgm_coag_icrp\\avg_parent_degrees.pdf"